In [ ]:
!pip install --quiet --requirement requirements.txt

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import urllib.request

from functions import read_levels_file, fft_analysis

In [ ]:
sns.set_theme()
%matplotlib inline

Download data from https://check-for-flooding.service.gov.uk/river-and-sea-levels?lng=-0.58408&lat=52.49929

https://riverlevels.uk/hebden-water-hebden-royd-nutclough

http://environment.data.gov.uk/flood-monitoring/id/stations/L1231/readings.csv?parameter=level&since=2010-01-01&_limit=10000&_sorted

In [ ]:
with urllib.request.urlopen('http://environment.data.gov.uk/flood-monitoring/id/stations/L1231/readings.csv?parameter=level&since=2010-01-01&_limit=10000&_sorted') as response:
    raw_data = [r.decode('utf-8') for r in response.readlines()]
    def reshape(row):
        x = row.split(',')
        return ','.join([x[0], x[2]])
    with open('L1231.csv', 'w') as csv_file:
        csv_file.write(reshape(raw_data[0]))
        csv_file.write(''.join([reshape(x) for x in raw_data[:1:-1]]))

In [ ]:
# RIVER_DATA='Nutclough-height-data-20220201-1130.csv'
RIVER_DATA='Nutclough-height-data-20220203-1300.csv'
# RIVER_DATA='readings-nutclough-2021-02-08.csv'
RIVER_DATA='L1231.csv'
# RIVER_DATA='Brearley-height-data.csv'

In [ ]:
(dates, levels) = read_levels_file(RIVER_DATA)

In [ ]:
ts = 15 * 60 # Sampling is 15 minutes
fs = 1.0 / ts
fft_size=50
result = fft_analysis(levels, fs, fft_size)

In [ ]:
plt.figure(figsize = (12, 6))
plt.plot(dates, levels, 'r')     
plt.xticks(rotation=30, ha='right')
plt.xlabel('Date')
plt.ylabel('Level')
plt.savefig('output/levels.png')
plt.show()

In [ ]:
plt.figure(figsize = (12, 6))
plt.imshow(np.transpose(result), origin='lower', cmap='jet', interpolation='nearest', aspect='auto')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time')
plt.xticks([])
plt.ticklabel_format(scilimits=[1,-1])
num_ticks = 5
plt.yticks(np.arange(num_ticks) * fft_size/(num_ticks-1), [f"{x:.1e}" for x in np.arange(num_ticks) * fs/2/(num_ticks-1)])
plt.savefig('output/fft.png')
plt.show()